# This is a Sátiro Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
import pandas as pd
import glob

# Ruta a los archivos CSV del dataset
ruta_archivos = 'data/*.csv'

# Cargar todos los archivos CSV en un solo DataFrame
archivos = glob.glob(ruta_archivos)
dataframes = [pd.read_csv(archivo) for archivo in archivos]
df = pd.concat(dataframes, ignore_index=True)

# Mostrar información básica del dataset
print(df.info())
print(df.head())
# Eliminar columnas irrelevantes (como Timestamp si existe)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [5]:
print("Dimensiones del dataset:", df.shape)
print("Nombres de columnas:", df.columns)

Dimensiones del dataset: (2830743, 79)
Nombres de columnas: Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean

In [6]:
# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Verificar que la columna 'Label' exista
if 'Label' not in df.columns:
    raise ValueError("La columna 'Label' no existe en el DataFrame.")

# Inspeccionar valores únicos en 'Label'
print("Valores únicos en Label:", df['Label'].unique())

# Limpiar valores si es necesario (errores tipográficos)
df['Label'] = df['Label'].str.strip()

# Convertir etiquetas a binario: 1 = ataque, 0 = benigno
df['Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)

# Verificar características
X = df.drop(columns=['Label'])  # Características
y = df['Label']  # Etiquetas

# Manejar valores problemáticos en X
import numpy as np

# Reemplazar valores infinitos por NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Rellenar valores NaN con la mediana de cada columna
X.fillna(X.median(), inplace=True)

# Validar que no existan más valores problemáticos
assert not X.isnull().any().any(), "Todavía hay valores NaN en las características."
assert not (X == float('inf')).any().any(), "Todavía hay valores infinitos en las características."

# Normalización
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Confirmación final
print("Normalización completada. Dimensiones de X_scaled:", X_scaled.shape)


Valores únicos en Label: ['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']
Normalización completada. Dimensiones de X_scaled: (2830743, 78)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

# Configurar el modelo con warm_start=True
rf_model = RandomForestClassifier(n_estimators=1, warm_start=True, random_state=42)

# Inicializar la barra de progreso
n_estimators = 100  # Número total de árboles
progress_bar = tqdm(total=n_estimators, desc="Entrenando Random Forest")

# Entrenar modelo árbol por árbol
for i in range(1, n_estimators + 1):
    rf_model.set_params(n_estimators=i)  # Incrementar el número de árboles
    rf_model.fit(X_train, y_train)  # Entrenar
    progress_bar.update(1)  # Actualizar barra de progreso

# Cerrar barra de progreso
progress_bar.close()

# Predicción en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluación del modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Entrenando Random Forest: 100%|██████████| 100/100 [15:44<00:00,  9.44s/it]


Accuracy: 0.9990614950372282
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    681572
           1       1.00      1.00      1.00    167651

    accuracy                           1.00    849223
   macro avg       1.00      1.00      1.00    849223
weighted avg       1.00      1.00      1.00    849223

Confusion Matrix:
 [[681093    479]
 [   318 167333]]


In [9]:
# Convertir las etiquetas en categorías numéricas
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])


In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Codificar etiquetas
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Predecir etiquetas con el modelo
y_pred = rf_model.predict(X_test)

# Convertir las clases a cadenas de texto
target_names = [str(cls) for cls in label_encoder.classes_]

# Generar el reporte de clasificación
print("Classification Report (Multiclass):\n", classification_report(y_test, y_pred, target_names=target_names))

Classification Report (Multiclass):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    681572
           1       1.00      1.00      1.00    167651

    accuracy                           1.00    849223
   macro avg       1.00      1.00      1.00    849223
weighted avg       1.00      1.00      1.00    849223



In [12]:
import joblib

# Guardar el modelo

joblib.dump(rf_model, 'models/random_forest_model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')

print("Modelo y escalador guardados correctamente.")


Modelo y escalador guardados correctamente.
